In [2]:
# Copyright (c) Facebook, Inc. and its affiliates.
# Modified by Bowen Cheng from: https://github.com/facebookresearch/detectron2/blob/master/demo/demo.py
import argparse
import glob
import multiprocessing as mp
import os

# fmt: off
import sys
sys.path.insert(1, os.path.join(sys.path[0], '..'))
# fmt: on

import tempfile
import time
import warnings

import cv2
import numpy as np
import tqdm

from detectron2.config import get_cfg
from detectron2.data.detection_utils import read_image
from detectron2.projects.deeplab import add_deeplab_config
from detectron2.utils.logger import setup_logger

from mask2former import add_maskformer2_config
from predictor import VisualizationDemo


# constants
WINDOW_NAME = "mask2former demo"


def setup_cfg(args):
    # load config from file and command-line arguments
    cfg = get_cfg()
    add_deeplab_config(cfg)
    add_maskformer2_config(cfg)
    cfg.merge_from_file(args.config_file)
    cfg.merge_from_list(args.opts)
    cfg.freeze()
    return cfg


def get_parser():
    parser = argparse.ArgumentParser(description="maskformer2 demo for builtin configs")
    parser.add_argument(
        "--config-file",
        default="configs/coco/panoptic-segmentation/maskformer2_R50_bs16_50ep.yaml",
        metavar="FILE",
        help="path to config file",
    )
    parser.add_argument("--webcam", action="store_true", help="Take inputs from webcam.")
    parser.add_argument("--video-input", help="Path to video file.")
    parser.add_argument(
        "--input",
        nargs="+",
        help="A list of space separated input images; "
        "or a single glob pattern such as 'directory/*.jpg'",
    )
    parser.add_argument(
        "--output",
        help="A file or directory to save output visualizations. "
        "If not given, will show output in an OpenCV window.",
    )

    parser.add_argument(
        "--confidence-threshold",
        type=float,
        default=0.5,
        help="Minimum score for instance predictions to be shown",
    )
    parser.add_argument(
        "--opts",
        help="Modify config options using the command-line 'KEY VALUE' pairs",
        default=[],
        nargs=argparse.REMAINDER,
    )
    return parser


def test_opencv_video_format(codec, file_ext):
    with tempfile.TemporaryDirectory(prefix="video_format_test") as dir:
        filename = os.path.join(dir, "test_file" + file_ext)
        writer = cv2.VideoWriter(
            filename=filename,
            fourcc=cv2.VideoWriter_fourcc(*codec),
            fps=float(30),
            frameSize=(10, 10),
            isColor=True,
        )
        [writer.write(np.zeros((10, 10, 3), np.uint8)) for _ in range(30)]
        writer.release()
        if os.path.isfile(filename):
            return True
        return False


if __name__ == "__main__":
    mp.set_start_method("spawn", force=True)
    args = get_parser().parse_args()
    setup_logger(name="fvcore")
    logger = setup_logger()
    logger.info("Arguments: " + str(args))

    cfg = setup_cfg(args)

    demo = VisualizationDemo(cfg)

    if args.input:
        if len(args.input) == 1:
            args.input = glob.glob(os.path.expanduser(args.input[0]))
            assert args.input, "The input path(s) was not found"
        for path in tqdm.tqdm(args.input, disable=not args.output):
            # use PIL, to be consistent with evaluation
            img = read_image(path, format="BGR")
            start_time = time.time()
            predictions, visualized_output = demo.run_on_image(img)
            logger.info(
                "{}: {} in {:.2f}s".format(
                    path,
                    "detected {} instances".format(len(predictions["instances"]))
                    if "instances" in predictions
                    else "finished",
                    time.time() - start_time,
                )
            )

            if args.output:
                if os.path.isdir(args.output):
                    assert os.path.isdir(args.output), args.output
                    out_filename = os.path.join(args.output, os.path.basename(path))
                else:
                    assert len(args.input) == 1, "Please specify a directory with args.output"
                    out_filename = args.output
                visualized_output.save(out_filename)
            else:
                cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
                cv2.imshow(WINDOW_NAME, visualized_output.get_image()[:, :, ::-1])
                if cv2.waitKey(0) == 27:
                    break  # esc to quit
    elif args.webcam:
        assert args.input is None, "Cannot have both --input and --webcam!"
        assert args.output is None, "output not yet supported with --webcam!"
        cam = cv2.VideoCapture(0)
        for vis in tqdm.tqdm(demo.run_on_video(cam)):
            cv2.namedWindow(WINDOW_NAME, cv2.WINDOW_NORMAL)
            cv2.imshow(WINDOW_NAME, vis)
            if cv2.waitKey(1) == 27:
                break  # esc to quit
        cam.release()
        cv2.destroyAllWindows()
    elif args.video_input:
        video = cv2.VideoCapture(args.video_input)
        width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
        frames_per_second = video.get(cv2.CAP_PROP_FPS)
        num_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
        basename = os.path.basename(args.video_input)
        codec, file_ext = (
            ("x264", ".mkv") if test_opencv_video_format("x264", ".mkv") else ("mp4v", ".mp4")
        )
        if codec == ".mp4v":
            warnings.warn("x264 codec not available, switching to mp4v")
        if args.output:
            if os.path.isdir(args.output):
                output_fname = os.path.join(args.output, basename)
                output_fname = os.path.splitext(output_fname)[0] + file_ext
            else:
                output_fname = args.output
            assert not os.path.isfile(output_fname), output_fname
            output_file = cv2.VideoWriter(
                filename=output_fname,
                # some installation of opencv may not support x264 (due to its license),
                # you can try other format (e.g. MPEG)
                fourcc=cv2.VideoWriter_fourcc(*codec),
                fps=float(frames_per_second),
                frameSize=(width, height),
                isColor=True,
            )
        assert os.path.isfile(args.video_input)
        for vis_frame in tqdm.tqdm(demo.run_on_video(video), total=num_frames):
            if args.output:
                output_file.write(vis_frame)
            else:
                cv2.namedWindow(basename, cv2.WINDOW_NORMAL)
                cv2.imshow(basename, vis_frame)
                if cv2.waitKey(1) == 27:
                    break  # esc to quit
        video.release()
        if args.output:
            output_file.release()
        else:
            cv2.destroyAllWindows()


/home/apd/miniconda3/envs/NNIS/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'predictor'

In [1]:
# Step 4: Import necessary libraries
import os
import numpy as np
from PIL import Image
import json
from tqdm import tqdm
import cv2
import sys
sys.path.append(r'C:\Users\absolute-zero\Desktop\NNIS\examples\Mask2Former')


In [2]:
# Step 5: Create directories for data
os.makedirs('data/train_images', exist_ok=True)
os.makedirs('data/val_images', exist_ok=True)
os.makedirs('data/annotations', exist_ok=True)


In [3]:
# Step 6: Generate synthetic dataset
def create_synthetic_dataset(num_images, image_dir, annotation_file):
    images = []
    annotations = []
    annotation_id = 1

    for img_id in tqdm(range(1, num_images + 1)):
        # Create a random image
        image = np.random.randint(0, 255, (256, 256, 3), dtype=np.uint8)
        img = Image.fromarray(image)
        img_filename = f'image_{img_id}.png'
        img.save(os.path.join(image_dir, img_filename))

        # Create a random mask with a simple shape
        mask = np.zeros((256, 256), dtype=np.uint8)
        cv2.rectangle(mask, (60, 60), (196, 196), 1, -1)  # Draw a filled rectangle
        mask_image = Image.fromarray(mask * 255)
        mask_filename = f'mask_{img_id}.png'
        mask_image.save(os.path.join(image_dir, mask_filename))

        # Create segmentation polygon for the rectangle
        segmentation = [[60, 60, 196, 60, 196, 196, 60, 196]]
        area = 136 * 136  # Area of the rectangle
        bbox = [60, 60, 136, 136]  # x, y, width, height

        # Image info
        images.append({
            'file_name': img_filename,
            'height': 256,
            'width': 256,
            'id': img_id
        })

        # Annotation info
        annotations.append({
            'id': annotation_id,
            'image_id': img_id,
            'category_id': 1,
            'segmentation': segmentation,
            'area': area,
            'bbox': bbox,
            'iscrowd': 0
        })

        annotation_id += 1

    # Categories
    categories = [{
        'id': 1,
        'name': 'rectangle',
        'supercategory': 'shape'
    }]

    # Create annotation file in COCO format
    annotation_data = {
        'images': images,
        'annotations': annotations,
        'categories': categories
    }

    with open(annotation_file, 'w') as f:
        json.dump(annotation_data, f)

# Create training data~
create_synthetic_dataset(10, 'data/train_images', 'data/annotations/instances_train.json')

# Create validation data
create_synthetic_dataset(2, 'data/val_images', 'data/annotations/instances_val.json')


100%|██████████| 2/2 [00:00<00:00, 118.56it/s]


In [4]:
# Step 7: Register datasets with Detectron2
from detectron2.data.datasets import register_coco_instances

register_coco_instances("synthetic_train", {}, "data/annotations/instances_train.json", "data/train_images")
register_coco_instances("synthetic_val", {}, "data/annotations/instances_val.json", "data/val_images")


/home/apd/miniconda3/envs/NNIS/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Step 8: Configure the Mask2Former model
import detectron2
from detectron2.config import get_cfg
from mask2former import add_maskformer2_config
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
add_maskformer2_config(cfg)

cfg.merge_from_file("/home/apd/NNIS/Mask2Former/configs/coco/instance-segmentation/maskformer2_R50_bs16_50ep.yaml")

cfg.DATASETS.TRAIN = ("synthetic_train",)
cfg.DATASETS.TEST = ("synthetic_val",)
cfg.DATALOADER.NUM_WORKERS = 1

cfg.MODEL.SEM_SEG_HEAD.NUM_CLASSES = 1
cfg.SOLVER.IMS_PER_BATCH = 2
cfg.SOLVER.BASE_LR = 0.00025
cfg.SOLVER.MAX_ITER = 1000

cfg.OUTPUT_DIR = "./output_synthetic"

cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)


KeyError: 'Non-existent config key: MODEL.RESNETS.STEM_TYPE'

In [ ]:
# Step 9: Train the model
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()


In [ ]:
# Step 10: Save the model path
model_weights_path = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
print(f"Model weights saved to: {model_weights_path}")


In [ ]:
# Step 11: Run inference to verify the model
import cv2
from detectron2.engine import DefaultPredictor
from detectron2.utils.visualizer import Visualizer

cfg.MODEL.WEIGHTS = model_weights_path
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.DATASETS.TEST = ("synthetic_val", )
cfg.MODEL.DEVICE = "cpu"

predictor = DefaultPredictor(cfg)

image = cv2.imread("data/val_images/image_1.png")
outputs = predictor(image)

v = Visualizer(image[:, :, ::-1], scale=1.0)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
result_image = v.get_image()[:, :, ::-1]

import matplotlib.pyplot as plt

plt.figure(figsize=(8, 8))
plt.imshow(result_image)
plt.axis('off')
plt.show()
